#                                         **1. CARGAR Y VISUALIZAR DEM PARA AOI**

Descripción: Permite cargar un shapefile local como área de interes para recortar un Modelo Digital de Elevación (DEM) de Google Earth Engine, visualizando un resultado interactivo con **geemap**, calcular estadisticas de elevación y exportarlo a la cuenta de google drive

## 1.1 Importar librerias necesarias

In [2]:
# Importar librerías necesarias
import ee
import geemap
import geopandas as gpd
from shapely.geometry import mapping

# Autenticación e inicialización (modo notebook y proyecto específico)
ee.Authenticate(auth_mode='notebook')

# Initialize the library.
ee.Initialize(project='ee-coberturap')

## 1.2 Cargar shapefile local

In [3]:
# Cargar shapefile local
shapefile_path = '/notebooks/ProyectoFinal/Shapes/'
gdf = gpd.read_file(shapefile_path)

# Obtener la geometría (primer polígono del shapefile)
geometry = ee.Geometry(mapping(gdf.geometry[0]))

# Cargar y recortar imagen DEM
dem = ee.Image('NASA/NASADEM_HGT/001').select('elevation')
dem_clip = dem.clip(geometry)

## 1.3 Estilos para visualización del DEM

In [4]:
# Estilos para visualización del DEM
dem_vis_params = {
    'min': 55,
    'max': 181,
    'palette': ['#00008B', '#0000CD', '#0000FF', '#00BFFF', '#ADD8E6', '#FFFFFF', '#FFFF00', '#FFA500', '#FF4500', '#8B0000']
}

# Crear mapa interactivo con geemap
Map = geemap.Map(center=[gdf.geometry[0].centroid.y, gdf.geometry[0].centroid.x], zoom=9)
Map.addLayer(dem_clip, dem_vis_params, 'DEM recortado')

## 1.4 Convertir GeoDataFrame a ee.FeatureCollection

Convierte el GeoDataFrame que contiene el shapefile en un *ee.FeatureCollection*. Necesario para que Earth Engine pueda procesar y visualizar esta geometría en su entorno.

In [5]:
# Convertir GeoDataFrame a ee.FeatureCollection

aoi_ee = geemap.gdf_to_ee(gdf)

# Ahora agrega el ee.FeatureCollection al mapa

aoi_vis_params = {
    'color': 'red',  # Ejemplo: color rojo para el contorno
    'width': 2,      # Ancho del borde
    'fillColor': '00000000' # Fondo transparente (ARGB, '00' para alfa transparente)
}
Map.addLayer(aoi_ee, aoi_vis_params, 'AOI shapefile (EE FeatureCollection)')


Map.addLayerControl()
Map


Map(center=[-3.439523286382691, -70.1442113276866], controls=(WidgetControl(options=['position', 'transparent_…

## 1.5 Identificación de valores minimos y maximos de elevación para el área

Los valores aquí extraidos se emplearan para ajustar el apartado "*1.3 estilos de visulización*" mejorando la visualización de los datos de elevación para el área de interes

In [6]:
min_val = dem_clip.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=geometry,
    scale=30, # Usar la misma escala que el DEM
    maxPixels=1e9 # Aumentar si el área es muy grande
).get('elevation').getInfo()

max_val = dem_clip.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=geometry,
    scale=30,
    maxPixels=1e9
).get('elevation').getInfo()

print(f"Valores de elevación en el área: Mínimo={min_val}m, Máximo={max_val}m")

Valores de elevación en el área: Mínimo=55m, Máximo=181m


## 1.6 Exportar imagen resultante a Google Drive

In [8]:
# Export the composite image to Google Drive
task = ee.batch.Export.image.toDrive(
    image=dem_clip,
    description='DEM',
    folder='earthengine',            # Destination folder in Google Drive
    fileNamePrefix='DEM', # File name without extension
    region=aoi_ee.geometry(),                   # Area to export
    ##scale=25,                        # Set a very high limit to allow exporting large images
                                     # Earth Engine requires this to avoid exporting by mistake very large rasters
                                     # The default limit is 1e8 (100 million pixels), so we override it here

    maxPixels=1e13,                  # Allow large exports
    fileFormat='GeoTIFF'
)

# Start the export task
task.start()

print("Export task started. Monitor progress in the Earth Engine Tasks tab.")

Export task started. Monitor progress in the Earth Engine Tasks tab.
